# export canditate model

In [ ]:
import sys
import tensorflow as tf
import keras 

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`
import dataset

# load dataset
train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"
train_ds, val_ds, nclass = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, 64
)


In [ ]:
trial_id = 295
candidate_model = f"saved_models/exp4_with_aug/best_model_t{trial_id}/"

model = keras.models.load_model(candidate_model)
model.load_weights(candidate_model + "weights/")
model.evaluate(val_ds)

model.summary()

# remove augmentation and regularization layers

In [ ]:

x = x_in = keras.layers.Input(shape=(256,256,3))

for l in model.layers:
    if isinstance(l, keras.layers.Conv2D):
        x = l(x)
    if isinstance(l, keras.layers.DepthwiseConv2D):
        x = l(x)
    if isinstance(l, keras.layers.Activation):
        x = l(x)
    if isinstance(l, keras.layers.Flatten):
        x = l(x)
    if isinstance(l, keras.layers.Dense):
        x = l(x)

striped_model = keras.models.Model(x_in, x)
striped_model.summary()

In [ ]:
striped_model.compile(
    "adam", loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
)

striped_model.evaluate(val_ds)


In [ ]:
striped_model.save(f"saved_models/striped_models/{trial_id}/{trial_id}.h5")